In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="aml-quickstarts-139038")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ARDSHADE5 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-139501
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-139501


In [1]:
!pip install -U scikit-learn==0.22.1

     |████████████████████████████████| 7.0 MB 6.3 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = 'conda_dependencies.yml')

In [3]:
import sklearn
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

In [20]:
#from azureml.core.environment import Environment
#from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
#env = Environment('tutorial-env')
#cd = CondaDependencies.create(pip_packages=['azureml-dataprep[pandas,fuse]>=1.1.14', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

#env.python.conda_dependencies = cd

# Register environment to re-use later
#env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_target_ws = ws.compute_targets["compute-cluster"]

compute_cluster = "compute-cluster"

try:
    compute_target_ws = ComputeTarget(workspace=ws, name=compute_cluster)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target_ws = ComputeTarget.create (ws, compute_cluster, compute_config)
    


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig 
from azureml.core import Environment
import os

# Specify parameter sampler
#https://www.kaggle.com/joparga3/2-tuning-parameters-for-logistic-regression
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
ps = RandomParameterSampling( {
    "--C": choice(0.001,0.01,0.1,1,10,100),
    "--max_iter": choice(50, 100, 150, 200)    
   })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
##est = ### YOUR CODE HERE ###

# create or retrieve an environment
#env = Environment.get(ws, name='MyEnvironment')

src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target= compute_target_ws,
                            environment=sklearn_env)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=12,
                                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_445a73e1-f8e1-4684-8f6f-449122f46a70,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
run.wait_for_completion(show_output=True)

RunId: HD_445a73e1-f8e1-4684-8f6f-449122f46a70
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_445a73e1-f8e1-4684-8f6f-449122f46a70?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-139501/workspaces/quick-starts-ws-139501

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-26T10:42:18.483522][API][INFO]Experiment created<END>\n""<START>[2021-02-26T10:42:19.319265][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-02-26T10:42:19.186424][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-26T10:42:20.1007058Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [9]:
import joblib
from azureml.widgets import RunDetails
# Get your best run and save the model from that run.
# https://github.com/microsoft/MLHyperparameterTuning/blob/f28c893f99ca22e32b0351a5e88873a2124afbd3/05_Train_Best_Model.ipynb

best_run = run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_445a73e1-f8e1-4684-8f6f-449122f46a70_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
pd.Series(best_parameters, name='Value').to_frame()



,Value
--C,1
--max_iter,200


In [11]:
#register the model
model = best_run.register_model(model_name='model', model_path='./outputs/model.pkl',tags=best_run.get_metrics())

In [12]:
#Save the best model
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_811c516192dd5b7a83a03c034db01b3e9ae72638bc95b24780b8d518dcd12a47_p.txt',
 'azureml-logs/65_job_prep-tvmps_811c516192dd5b7a83a03c034db01b3e9ae72638bc95b24780b8d518dcd12a47_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_811c516192dd5b7a83a03c034db01b3e9ae72638bc95b24780b8d518dcd12a47_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/107_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.pkl']

In [13]:
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

## AutoML experiment

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [15]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [16]:
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [17]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

In [18]:
y = y.to_frame()
y

,y
0,0
1,0
2,0
3,0
4,0
...,...
32945,0
32946,0
32947,0
32948,0


In [19]:
Full_data = pd.concat([x, y], axis=1)
Full_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [20]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(Full_data).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)

 

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing-all-features.ipynb

#Primary metric: https://docs.microsoft.com/en-us/azure/machine-learning/algorithm-module-reference/tune-model-hyperparameters

'''
Metrics used for binary classification
Accuracy is the proportion of true results to total cases.
Precision is the proportion of true results to positive results.
Recall is the fraction of all correct results over all results.
F-score is a measure that balances precision and recall.
AUC is a value that represents the area under the curve when false positives are plotted on the x-axis and true positives are plotted on the y-axis.
Average Log Loss is the difference between two probability distributions: the true one, and the one in the model.

Metrics used for regression
Mean absolute error averages all the errors in the model, where error means the distance of the predicted value from the true value. It's often abbreviated as MAE.
Root of mean squared error measures the average of the squares of the errors, and then takes the root of that value. It's often abbreviated as RMSE.
Relative absolute error represents the error as a percentage of the true value.
Relative squared error normalizes the total squared error by dividing by the total squared error of the predicted values.
Coefficient of determination is a single number that indicates how well data fits a model. A value of one means that the model exactly matches the data. A value of zero means that the data is random or otherwise can't be fit to the model. It's often called r2, R2, or r-squared.
'''

automl_settings = {
    "experiment_timeout_hours" : 0.5,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}


automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target='compute-cluster',
                             training_data = train_data,
                             label_column_name = label,
                             **automl_settings
                            )

In [22]:
# Submit your automl run

automl_run = exp.submit(automl_config ,show_output=True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_42bcdb2d-71ce-4b0f-b106-b526d488c85b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because t

In [23]:

# Wait for the remote run to complete
automl_run.wait_for_completion()

{'runId': 'AutoML_42bcdb2d-71ce-4b0f-b106-b526d488c85b',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-26T11:05:03.374926Z',
 'endTimeUtc': '2021-02-26T11:28:07.970352Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-139501","workspace_name":"quick-starts-ws-139501","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_cl

In [24]:
# Retrieve and save your best automl model.

best_run_auto, best_model_auto = automl_run.get_output()
best_model_auto

Package:azureml-automl-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-core, training version:1.22.0, current version:1.20.0
Package:azureml-dataprep, training version:2.9.1, current version:2.7.3
Package:azureml-dataprep-native, training version:29.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.7.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-defaults, training version:1.22.0, current version:1.20.0
Package:azureml-interpret, training version:1.22.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.22.0, current version:1.20.0
Package:azureml-telemetry, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.22.0, current version:1.20.0


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.42105263157894735,
                                                                                                silent=True,
                                                                                                subsample=0.05,
                                           

In [30]:
#save model as pickle

model_auto = best_run_auto.register_model(model_name='model_auto', model_path='./outputs',tags=best_run_auto.get_metrics())

Current provisioning state of AmlCompute is "Deleting"

